In [180]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import json
import pickle
import random
import time
import math
import numpy as np
import copy

torch.manual_seed(1)

# 데이터 준비

In [39]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [2]:
data = open('../../dataset/corpus/bAbI/en/qa14_time-reasoning_train.txt','r',encoding='utf-8').readlines()
data = [d[:-1] for d in data]

In [173]:
len(data)

3372

In [181]:
train=[]
support=[]
qa=[]

In [182]:
for d in data:

    index = d.split(' ')[0]
    if index=='1':
        support=[]
        qa=[]
        
    if '?' in d:
        temp = d.split('\t')
        q = temp[0].strip().replace('?','').split(' ')[1:]
        a = temp[1]
        stemp = copy.deepcopy(support)
        train.append([stemp,(q,a)])
    else:
        tokens = d.replace('.','').split(' ')[1:]
        support.append(tokens)

In [183]:
len(train)

1000

In [184]:
word_to_ix={}
target_to_ix={}

for t in train:
    supports = t[0]
    q = t[1][0]
    a = t[1][1]
    
    s = flatten(supports)
    for word in s + q:
        if word not in list(word_to_ix.keys()):
            word_to_ix[word]=len(word_to_ix)
    
    if a not in list(target_to_ix.keys()):
        target_to_ix[a] = len(target_to_ix)
    
ix_to_word = {v:k for k,v in word_to_ix.items()}

In [40]:
def prepare_sequence(seq):
    idxs = list(map(lambda w: word_to_ix[w], seq))
    tensor = torch.LongTensor(idxs)
    return Variable(tensor)

In [233]:
class RelationalNetwork(nn.Module):
    def __init__(self, input_size,target_size,hidden_size, n_layers=1):
        super(RelationalNetwork, self).__init__()
        
        self.input_size = input_size
        self.target_size = target_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm_o = nn.LSTM(hidden_size, hidden_size,num_layers=n_layers) # for objects
        self.lstm_q = nn.LSTM(hidden_size, hidden_size,num_layers=n_layers) # for question (separate LSTM)
        
        self.relu = nn.ReLU()
        self.reaky = nn.LeakyReLU()
        
        # g
        self.g_1 = nn.Linear(hidden_size*3,128) # obj+obj+question 96
        self.g_2 = nn.Linear(128,64)
        #self.g_3 = nn.Linear(128,128) # because data is much smaller compared with paper.
        
        # f
        self.f_1 = nn.Linear(64,32)
        self.f_2 = nn.Linear(32,target_size)
        
    def init_hidden(self):
        return (Variable( torch.randn(self.n_layers, 1, self.hidden_size)),
                 Variable( torch.randn(self.n_layers, 1, self.hidden_size)) )
    
    def forward(self, supports, question):
        
        # question embedding
        hidden_q = self.init_hidden()
        embedded = self.embedding(question)
        output, hidden_q = self.lstm_q(embedded.view(len(question),1,-1),hidden_q)
        
        q_embed = hidden_q[0]
        
        # object embedding
        obj=[]
        for support in supports:
            hidden_o = self.init_hidden()
            embedded = self.embedding(support)
            output, hidden_o = self.lstm_o(embedded.view(len(support),1,-1), hidden_o)
            obj.append(hidden_o[0])
        
        # g
        x_g_sum=0
        for o_i in range(len(obj)):
            for o_j in range(len(obj)):
                
                if o_i==o_j: continue
                
                x_g = torch.cat((obj[o_i],obj[o_j],q_embed),2)
                x_g = x_g.squeeze(1)
                x_g = self.relu(self.g_1(x_g))
                x_g = self.relu(self.g_2(x_g))
                #x_g = self.relu(self.g_3(x_g)) because data is much smaller compared with paper.
                x_g_sum+=x_g # element-wise sum
        
        # f
        x_f = self.relu(self.f_1(x_g_sum))
        x_f = self.f_2(x_f)
        
        out = F.log_softmax(x_f)
        
        return out

In [234]:
model = RelationNetwork(len(word_to_ix),len(target_to_ix),32)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001) # 논문에선 2*e^-4 .==. 0.036
losses=[]
STEP=5

In [235]:
for step in range(STEP):

    for index, (supports,qa) in enumerate(train):


        model.zero_grad()
        supports = list(map(lambda s:prepare_sequence(s),supports))
        question = prepare_sequence(qa[0])
        answer = target_to_ix[qa[1]]
        pred = model(supports,question)
        #print(pred)
        loss = loss_function(pred,Variable(torch.LongTensor([answer])))
        loss.backward()

        losses.append(loss.data.numpy()[0])
        torch.nn.utils.clip_grad_norm(model.parameters(), 5.0)

        optimizer.step()

        if index%100==0:
            #print(pred)
            print('STEP [%d/%d] EPOCH [%d/%d] LOSS %.4f' % (step+1,STEP,index,len(train),np.mean(losses)))
            losses=[]

STEP [1/5] EPOCH [0/1000] LOSS 1.6335
STEP [1/5] EPOCH [100/1000] LOSS 2.7066
STEP [1/5] EPOCH [200/1000] LOSS 2.3561
STEP [1/5] EPOCH [300/1000] LOSS 2.0963
STEP [1/5] EPOCH [400/1000] LOSS 1.9286


KeyboardInterrupt: 